### In Depth Analysis - Modeling & Hyper parameter tuning

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set()

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

from scipy import stats

from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import regularizers
from keras import optimizers
import tensorflow as tf
from keras import backend as K
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from skopt import BayesSearchCV

from sklearn.metrics import accuracy_score,classification_report, roc_auc_score

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
##all features
application_train = pd.read_csv('new_application_train_merged.csv', index_col=None, engine='python')
application_test = pd.read_csv('new_application_test_merged.csv', index_col=None, engine='python')


In [3]:
#top 20 features
new_application_train = pd.read_csv('application_train_for_ML.csv', index_col=None, engine='python')
new_application_test = pd.read_csv('application_test_for_ML.csv', index_col=None, engine='python')

## Feature Imputing & Feature Scaling 


In [4]:
new_application_train.shape

(43845, 21)

In [5]:
imputer = SimpleImputer(strategy="median")
scaler = MinMaxScaler(feature_range = (0, 1))

In [6]:
imputer.fit(new_application_train.drop(['TARGET'], axis=1))


SimpleImputer(strategy='median')

In [7]:
train_transformed = imputer.transform(new_application_train.drop(['TARGET'], axis=1))
train_transformed = scaler.fit_transform(train_transformed)


In [8]:
test_transformed = imputer.transform(new_application_test)
test_transformed = scaler.transform(test_transformed)


In [9]:
X_training_set, X_validation_set, y_training_set, y_validation_set = train_test_split(train_transformed, 
                                                                                      new_application_train['TARGET'], test_size=0.33, random_state=42)

In [10]:
def metric_calc(pred):
    print("The accuracy score : ", accuracy_score(y_validation_set,pred))
    print("The classification report is as follows:\n", classification_report(y_validation_set,pred))
    print("Taregt Values:")
    print(pd.DataFrame({'target':pred})['target'].value_counts())
    print("ROC AUC score is: ",roc_auc_score(y_validation_set,pred))

## 1. Logistic Regression

In [11]:
logistic_regressor = LogisticRegression(C = 2)
logistic_regressor.fit(X_training_set,y_training_set)
log_regression_pred = logistic_regressor.predict(X_validation_set)
pd.DataFrame({'target':log_regression_pred})['target'].value_counts()
metric_calc(log_regression_pred)

The accuracy score :  0.675236712972562
The classification report is as follows:
               precision    recall  f1-score   support

           0       0.65      0.61      0.63      6578
           1       0.69      0.73      0.71      7891

    accuracy                           0.68     14469
   macro avg       0.67      0.67      0.67     14469
weighted avg       0.67      0.68      0.67     14469

Taregt Values:
1    8322
0    6147
Name: target, dtype: int64
ROC AUC score is:  0.669814588391338


C:\Users\kneelagandan\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## 2. Random Forest 

In [12]:

random_forest = RandomForestClassifier(n_estimators = 500, random_state = 50, verbose = 1)

In [13]:
random_forest.fit(X_training_set,y_training_set)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:   24.6s finished


RandomForestClassifier(n_estimators=500, random_state=50, verbose=1)

In [14]:
random_forest_pred = random_forest.predict(X_validation_set)
metric_calc(random_forest_pred)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


The accuracy score :  0.6761351855691479
The classification report is as follows:
               precision    recall  f1-score   support

           0       0.65      0.61      0.63      6578
           1       0.69      0.73      0.71      7891

    accuracy                           0.68     14469
   macro avg       0.67      0.67      0.67     14469
weighted avg       0.68      0.68      0.68     14469

Taregt Values:
1    8293
0    6176
Name: target, dtype: int64
ROC AUC score is:  0.6709039116459788


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    1.5s finished


In [15]:
random_forest_pred_test = random_forest.predict_proba(test_transformed)
submission_random_forest = application_test[['SK_ID_CURR']]
submission_random_forest['TARGET'] = random_forest_pred_test[:,1]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    5.2s finished
<ipython-input-15-86f9992d35cb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_random_forest['TARGET'] = random_forest_pred_test[:,1]


## 2.1 Feature importance of random forest classifier


In [16]:
features = list(set(new_application_train.columns).difference({'TARGET'}))

feature_importance_df = pd.DataFrame({'Feature':features,'Importance':random_forest.feature_importances_})
df = feature_importance_df.sort_values(['Importance'],ascending=False).reset_index()
df = df.drop(['index'],axis=1)
df.head()

,Feature,Importance
0,AMT_GOODS_PRICE_x,0.172237
1,NAME_EDUCATION_TYPE,0.159289
2,FLAG_MOBIL,0.118375
3,CNT_FAM_MEMBERS,0.104598
4,EXT_SOURCE_3,0.086871


## 3. Naive Bayes Classifier


In [17]:
bayes_class = GaussianNB()
bayes_class.fit(X_training_set,y_training_set)
bayes_preds = bayes_class.predict(X_validation_set)
metric_calc(bayes_preds)

The accuracy score :  0.6520837652913125
The classification report is as follows:
               precision    recall  f1-score   support

           0       0.62      0.61      0.61      6578
           1       0.68      0.69      0.68      7891

    accuracy                           0.65     14469
   macro avg       0.65      0.65      0.65     14469
weighted avg       0.65      0.65      0.65     14469

Taregt Values:
1    8039
0    6430
Name: target, dtype: int64
ROC AUC score is:  0.6482590362863982


In [18]:
nb_pred_test = bayes_class.predict_proba(test_transformed)
submission_nb = application_test[['SK_ID_CURR']]
submission_nb['TARGET'] = nb_pred_test[:,1]


<ipython-input-18-3115073d959d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_nb['TARGET'] = nb_pred_test[:,1]


## 4 .XG Boost

In [19]:
xgb_classifier = XGBClassifier(objective ='binary:logistic')
xgb_classifier.fit(X_training_set,y_training_set)
xgb_pred = xgb_classifier.predict(X_validation_set)
metric_calc(xgb_pred)

The accuracy score :  0.6748220333125994
The classification report is as follows:
               precision    recall  f1-score   support

           0       0.65      0.61      0.63      6578
           1       0.69      0.73      0.71      7891

    accuracy                           0.67     14469
   macro avg       0.67      0.67      0.67     14469
weighted avg       0.67      0.67      0.67     14469

Taregt Values:
1    8336
0    6133
Name: target, dtype: int64
ROC AUC score is:  0.6693079322368055


In [20]:
xgb_pred_test = xgb_classifier.predict_proba(test_transformed)
submission_xgb = application_test[['SK_ID_CURR']]
submission_xgb['TARGET'] = xgb_pred_test[:,1]


<ipython-input-20-65d003118767>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_xgb['TARGET'] = xgb_pred_test[:,1]


## 5 . Ensemble Modeling 
https://quantdare.com/what-is-the-difference-between-bagging-and-boosting/

In [21]:
#Voting Approach
def stacked_model(X_training_set):
    """
    This method performs the stacked ensambling of all the models - Random forest, 
    Naive Bayes,Logistic Regression.
    """  
    stacked_predictions = np.array([])

    for element in X_training_set:
         stacked_predictions = np.append(stacked_predictions,stats.mode(element)[0][0])

    return stacked_predictions

Combine all the test results into a multidimensional array to feed into the stacked model

In [22]:
combined_array = (pd.DataFrame({'LR':log_regression_pred,
                                'RF':random_forest_pred,
                                'Bayes':bayes_preds,
                                'XGB':xgb_pred}).values)
stacked_model_pred = stacked_model(combined_array)
metric_calc(stacked_model_pred)

The accuracy score :  0.6807657751053977
The classification report is as follows:
               precision    recall  f1-score   support

           0       0.64      0.67      0.65      6578
           1       0.71      0.69      0.70      7891

    accuracy                           0.68     14469
   macro avg       0.68      0.68      0.68     14469
weighted avg       0.68      0.68      0.68     14469

Taregt Values:
1.0    7662
0.0    6807
Name: target, dtype: int64
ROC AUC score is:  0.6795632739153977


## Observation : Ensemble provides slight better result though not signifacnt improvement 

## 6. Neural Network 

In [23]:
model = Sequential()
new_application_test.shape[1]
model.add(Dense(100, input_dim=new_application_train.drop(['TARGET'], axis=1).shape[1], activation= "relu"))
model.add(Dense(50, activation= "relu"))
model.add(Dense(1,activation= "sigmoid"))
model.summary() #Print model Summary


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               2100      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 7,201
Trainable params: 7,201
Non-trainable params: 0
_________________________________________________________________


In [24]:
def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

In [25]:
model.compile(loss= "binary_crossentropy" , optimizer="adam", metrics=['accuracy', auroc])
model.fit(X_training_set, y_training_set, epochs=500)


Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    
Train on 29376 samples
Epoch 1/500
29376/29376 [==============================] - 2s 70us/sample - loss: 0.6230 - acc: 0.6539 - auroc: 0.7132
Epoch 2/500
29376/29376 [==============================] - 2s 68us/sample - loss: 0.6141 - acc: 0.6638 - auroc: 0.7228
Epoch 3/500
29376/29376 [==============================] - 2s 64us

29376/29376 [==============================] - 2s 73us/sample - loss: 0.5772 - acc: 0.6954 - auroc: 0.7628
Epoch 61/500
29376/29376 [==============================] - 2s 65us/sample - loss: 0.5771 - acc: 0.6919 - auroc: 0.7624
Epoch 62/500
29376/29376 [==============================] - 3s 90us/sample - loss: 0.5770 - acc: 0.6941 - auroc: 0.7618
Epoch 63/500
29376/29376 [==============================] - 2s 73us/sample - loss: 0.5762 - acc: 0.6939 - auroc: 0.7640
Epoch 64/500
29376/29376 [==============================] - 3s 100us/sample - loss: 0.5745 - acc: 0.6947 - auroc: 0.7649
Epoch 65/500
29376/29376 [==============================] - 4s 129us/sample - loss: 0.5744 - acc: 0.6959 - auroc: 0.7660
Epoch 66/500
29376/29376 [==============================] - 5s 162us/sample - loss: 0.5732 - acc: 0.6952 - auroc: 0.7664
Epoch 67/500
29376/29376 [==============================] - 3s 88us/sample - loss: 0.5733 - acc: 0.6955 - auroc: 0.7660
Epoch 68/500
29376/29376 [========================

29376/29376 [==============================] - 3s 115us/sample - loss: 0.5357 - acc: 0.7219 - auroc: 0.8008
Epoch 129/500
29376/29376 [==============================] - 3s 90us/sample - loss: 0.5356 - acc: 0.7247 - auroc: 0.8023
Epoch 130/500
29376/29376 [==============================] - 2s 83us/sample - loss: 0.5341 - acc: 0.7231 - auroc: 0.7998
Epoch 131/500
29376/29376 [==============================] - 2s 78us/sample - loss: 0.5332 - acc: 0.7251 - auroc: 0.8033
Epoch 132/500
29376/29376 [==============================] - 3s 99us/sample - loss: 0.5330 - acc: 0.7271 - auroc: 0.8026
Epoch 133/500
29376/29376 [==============================] - 3s 97us/sample - loss: 0.5322 - acc: 0.7280 - auroc: 0.8046
Epoch 134/500
29376/29376 [==============================] - 2s 79us/sample - loss: 0.5324 - acc: 0.7259 - auroc: 0.8039
Epoch 135/500
29376/29376 [==============================] - 3s 91us/sample - loss: 0.5305 - acc: 0.7236 - auroc: 0.8050
Epoch 136/500
29376/29376 [==================

29376/29376 [==============================] - 3s 109us/sample - loss: 0.5062 - acc: 0.7439 - auroc: 0.8269
Epoch 194/500
29376/29376 [==============================] - 3s 108us/sample - loss: 0.5049 - acc: 0.7404 - auroc: 0.8263
Epoch 195/500
29376/29376 [==============================] - 3s 116us/sample - loss: 0.5067 - acc: 0.7420 - auroc: 0.8260
Epoch 196/500
29376/29376 [==============================] - 4s 141us/sample - loss: 0.5044 - acc: 0.7435 - auroc: 0.8280
Epoch 197/500
29376/29376 [==============================] - 4s 120us/sample - loss: 0.5049 - acc: 0.7424 - auroc: 0.8258
Epoch 198/500
29376/29376 [==============================] - 3s 108us/sample - loss: 0.5041 - acc: 0.7420 - auroc: 0.8281
Epoch 199/500
29376/29376 [==============================] - 4s 136us/sample - loss: 0.5042 - acc: 0.7420 - auroc: 0.8264
Epoch 200/500
29376/29376 [==============================] - 4s 130us/sample - loss: 0.5050 - acc: 0.7413 - auroc: 0.8269
Epoch 201/500
29376/29376 [===========

29376/29376 [==============================] - 3s 108us/sample - loss: 0.4867 - acc: 0.7528 - auroc: 0.8387
Epoch 261/500
29376/29376 [==============================] - 3s 94us/sample - loss: 0.4874 - acc: 0.7530 - auroc: 0.8397
Epoch 262/500
29376/29376 [==============================] - 3s 91us/sample - loss: 0.4847 - acc: 0.7573 - auroc: 0.8414
Epoch 263/500
29376/29376 [==============================] - 4s 121us/sample - loss: 0.4857 - acc: 0.7555 - auroc: 0.8406
Epoch 264/500
29376/29376 [==============================] - 3s 114us/sample - loss: 0.4841 - acc: 0.7551 - auroc: 0.8425
Epoch 265/500
29376/29376 [==============================] - 3s 97us/sample - loss: 0.4854 - acc: 0.7556 - auroc: 0.8404
Epoch 266/500
29376/29376 [==============================] - 3s 92us/sample - loss: 0.4857 - acc: 0.7544 - auroc: 0.8420
Epoch 267/500
29376/29376 [==============================] - 3s 100us/sample - loss: 0.4855 - acc: 0.7517 - auroc: 0.8405
Epoch 268/500
29376/29376 [===============

29376/29376 [==============================] - 2s 71us/sample - loss: 0.4723 - acc: 0.7618 - auroc: 0.8498
Epoch 327/500
29376/29376 [==============================] - 2s 65us/sample - loss: 0.4694 - acc: 0.7621 - auroc: 0.8529
Epoch 328/500
29376/29376 [==============================] - 2s 68us/sample - loss: 0.4717 - acc: 0.7606 - auroc: 0.8501
Epoch 329/500
29376/29376 [==============================] - 2s 79us/sample - loss: 0.4708 - acc: 0.7635 - auroc: 0.8498
Epoch 330/500
29376/29376 [==============================] - 2s 72us/sample - loss: 0.4718 - acc: 0.7610 - auroc: 0.8499
Epoch 331/500
29376/29376 [==============================] - 2s 77us/sample - loss: 0.4710 - acc: 0.7627 - auroc: 0.8508
Epoch 332/500
29376/29376 [==============================] - 2s 69us/sample - loss: 0.4707 - acc: 0.7631 - auroc: 0.8510
Epoch 333/500
29376/29376 [==============================] - 2s 73us/sample - loss: 0.4704 - acc: 0.7633 - auroc: 0.8527
Epoch 334/500
29376/29376 [===================

29376/29376 [==============================] - 2s 74us/sample - loss: 0.4614 - acc: 0.7691 - auroc: 0.8592
Epoch 394/500
29376/29376 [==============================] - 2s 79us/sample - loss: 0.4597 - acc: 0.7693 - auroc: 0.8590
Epoch 395/500
29376/29376 [==============================] - 2s 77us/sample - loss: 0.4626 - acc: 0.7666 - auroc: 0.8571
Epoch 396/500
29376/29376 [==============================] - 2s 85us/sample - loss: 0.4591 - acc: 0.7697 - auroc: 0.8587
Epoch 397/500
29376/29376 [==============================] - 2s 79us/sample - loss: 0.4601 - acc: 0.7673 - auroc: 0.8590
Epoch 398/500
29376/29376 [==============================] - 2s 83us/sample - loss: 0.4615 - acc: 0.7685 - auroc: 0.8575
Epoch 399/500
29376/29376 [==============================] - 3s 105us/sample - loss: 0.4603 - acc: 0.7700 - auroc: 0.8587
Epoch 400/500
29376/29376 [==============================] - 2s 84us/sample - loss: 0.4596 - acc: 0.7685 - auroc: 0.8594
Epoch 401/500
29376/29376 [==================

29376/29376 [==============================] - 3s 86us/sample - loss: 0.4515 - acc: 0.7749 - auroc: 0.8641
Epoch 461/500
29376/29376 [==============================] - 3s 99us/sample - loss: 0.4512 - acc: 0.7747 - auroc: 0.8654
Epoch 462/500
29376/29376 [==============================] - 4s 145us/sample - loss: 0.4518 - acc: 0.7766 - auroc: 0.8651
Epoch 463/500
29376/29376 [==============================] - 5s 156us/sample - loss: 0.4511 - acc: 0.7739 - auroc: 0.8641
Epoch 464/500
29376/29376 [==============================] - 4s 141us/sample - loss: 0.4508 - acc: 0.7755 - auroc: 0.8653
Epoch 465/500
29376/29376 [==============================] - 4s 121us/sample - loss: 0.4507 - acc: 0.7743 - auroc: 0.8639
Epoch 466/500
29376/29376 [==============================] - 2s 81us/sample - loss: 0.4508 - acc: 0.7752 - auroc: 0.8648
Epoch 467/500
29376/29376 [==============================] - 2s 71us/sample - loss: 0.4511 - acc: 0.7739 - auroc: 0.8641
Epoch 468/500
29376/29376 [===============

In [26]:
pred= model.predict(X_validation_set)


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


In [27]:
print("ROC AUC score is: ",roc_auc_score(y_validation_set,pred))

ROC AUC score is:  0.6600490746931655


In [28]:
nn_pred_test = model.predict(test_transformed)
nn_pred_test_conv = [y[0] for y in nn_pred_test]


In [29]:
submission_nn = application_test[['SK_ID_CURR']]
submission_nn['TARGET'] = nn_pred_test_conv

<ipython-input-29-30b7dc9e3cc9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_nn['TARGET'] = nn_pred_test_conv


## Observation : With NN we can achieve 0.66 ROC-AUC score after 500 echoes

## 7.Hyper Parameter Tuning 

## 7.1 Logistic Regression - Grid Search CV

In [30]:
##Tuning C Value
logistic_regressor = LogisticRegression()

c_space = np.logspace(0.01,0.01,1) 
param_grid = {'C': c_space} 

logreg_cv = GridSearchCV(logistic_regressor, param_grid, cv = 5) 
logreg_cv.fit(X_training_set,y_training_set)


log_regression_pred = logreg_cv.predict(X_validation_set)

C:\Users\kneelagandan\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\kneelagandan\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

In [31]:
metric_calc(log_regression_pred)

The accuracy score :  0.6735088810560509
The classification report is as follows:
               precision    recall  f1-score   support

           0       0.65      0.61      0.63      6578
           1       0.69      0.73      0.71      7891

    accuracy                           0.67     14469
   macro avg       0.67      0.67      0.67     14469
weighted avg       0.67      0.67      0.67     14469

Taregt Values:
1    8321
0    6148
Name: target, dtype: int64
ROC AUC score is:  0.6680787338154288


In [32]:
log_regression_pred_test = logreg_cv.predict_proba(test_transformed)
log_regression_pred_test[:,1]
submission_log_regression = application_test[['SK_ID_CURR']]
submission_log_regression['TARGET'] = log_regression_pred_test[:,1]

<ipython-input-32-958f2f9d48fc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_log_regression['TARGET'] = log_regression_pred_test[:,1]


In [33]:
print("Best score is {}".format(logreg_cv.best_score_)) 


Best score is 0.6692197268368988


## Observation : ROC-AUC Score not much improved after tuning the C value - let's try with XG Boost Tuning

## 7.2 XG Boost - BayesSearchCV

In [34]:
# Classifier
bayes_cv_tuner = BayesSearchCV(
    estimator = XGBClassifier(
        n_jobs = 1,
        objective = 'binary:logistic',
        eval_metric = 'auc',
        silent=1,
        tree_method='approx'
    ),
    search_spaces = {
        'learning_rate': (0.01, 1.0, 'log-uniform'),
        'min_child_weight': (0, 10),
        'max_depth': (0, 50),
        'max_delta_step': (0, 20),
        'subsample': (0.01, 1.0, 'uniform'),
        'colsample_bytree': (0.01, 1.0, 'uniform'),
        'colsample_bylevel': (0.01, 1.0, 'uniform'),
        'reg_lambda': (1e-9, 1000, 'log-uniform'),
        'reg_alpha': (1e-9, 1.0, 'log-uniform'),
        'gamma': (1e-9, 0.5, 'log-uniform'),
        'min_child_weight': (0, 5),
        'n_estimators': (50, 100),
        'scale_pos_weight': (1e-6, 500, 'log-uniform')
    },    
    scoring = 'roc_auc',
    cv = StratifiedKFold(
        n_splits=3,
        shuffle=True,
        random_state=42
    ),
    n_jobs = 3,
    n_iter = 9,   
    verbose = 0,
    refit = True,
    random_state = 42
)

def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
    
    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))

In [35]:
result = bayes_cv_tuner.fit(X_training_set,y_training_set, callback=status_print)

Model #1
Best ROC-AUC: 0.5426
Best params: OrderedDict([('colsample_bylevel', 0.4160029192647807), ('colsample_bytree', 0.7304484857455519), ('gamma', 0.13031389926541354), ('learning_rate', 0.042815319280763466), ('max_delta_step', 13), ('max_depth', 21), ('min_child_weight', 2), ('n_estimators', 87), ('reg_alpha', 5.497557739289786e-07), ('reg_lambda', 648), ('scale_pos_weight', 275), ('subsample', 0.13556548021189216)])

Model #2
Best ROC-AUC: 0.7357
Best params: OrderedDict([('colsample_bylevel', 0.8390144719977516), ('colsample_bytree', 0.8844821246070537), ('gamma', 4.358684608480795e-07), ('learning_rate', 0.7988179462781242), ('max_delta_step', 17), ('max_depth', 3), ('min_child_weight', 1), ('n_estimators', 68), ('reg_alpha', 0.0005266983003701547), ('reg_lambda', 953), ('scale_pos_weight', 315), ('subsample', 0.9923710598637134)])

Model #3
Best ROC-AUC: 0.7357
Best params: OrderedDict([('colsample_bylevel', 0.8390144719977516), ('colsample_bytree', 0.8844821246070537), ('gam

In [36]:
# XGB parameters found by Bayesian optimization above
classifier = XGBClassifier(
            objective ='binary:logistic',
            colsample_bylevel= 0.8015579071911014, 
            colsample_bytree= 0.44364889457651413, 
            gamma= 3.811128976537413e-05, 
            learning_rate= 0.270039020618534, 
            max_delta_step= 18, 
            max_depth=36, 
            min_child_weight= 2, 
            n_estimators= 83, 
            reg_alpha= 1.5057560255472018e-06, 
            reg_lambda= 659, 
            scale_pos_weight= 256, 
            subsample= 0.8835665823899177)

#cv = StratifiedKFold(n_splits = 5, random_state = 2017)
#roc_auc=[]
#X=train_transformed
#y=new_application_train['TARGET'].values
#for train, test in cv.split(X,y):
    #probas_ = classifier.fit(X[train], y[train]).predict_proba(X[test])
    #roc_auc.append(roc_auc_score(y[test], probas_[:, 1]))
#print(np.mean(roc_auc))
xgb_classifier.fit(X_training_set,y_training_set)
xgb_pred = xgb_classifier.predict_proba(X_validation_set)
print("ROC AUC score is: ",roc_auc_score(y_validation_set,xgb_pred[:, 1]))

ROC AUC score is:  0.7319908964875989


## Observation : After Hyper Tuning XGB ROC-AUC Score improved from 0.666 to 0.731